## 1

In [1]:
package main
import "fmt"

func main() {
	ch := make(chan int, 1)
	ch <- 1
	ch <- 2
	fmt.Println(<-ch)
}

fatal error: all goroutines are asleep - deadlock!

goroutine 1 [chan send]:
main.main()
	 [[ Cell [1] Line 7 ]] /var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_f7e15773/main.go:11 +0x48
exit status 2


## 3

In [ ]:
package main
import "fmt"
import "time"

func main() {
	ch := make(chan int)
	go func() {
		for i := 0; i < 3; i++ {
			ch <- i 
		}
		close(ch)
	}()

	for{
		select {
		case v, ok := <- ch:
			if !ok {
				fmt.Println("done")
				return
			} 
			fmt.Println(v)

			case <- time.After( 50 * time.Millisecond):
				fmt.Println("timeout")
		}
	}
}

0
1
2
done


## 7

main結束了, process也結束了. => worker也結束了
=> 如果main沒有結束, 那就是goroutine leak ( worker)

In [10]:
package main
import "fmt"
import "time"

func worker(ch chan int) {
    for range ch {
        fmt.Println("Hello")
    }
}

func main() {
    ch := make(chan int)
    go worker(ch)
    time.Sleep(time.Second)
	fmt.Println("Awake")
}


Awake


## 10

In [ ]:
package main
import "fmt"

type Person struct {
	Name string
}

func (p *Person) updateName(newName string){
	p.Name = newName
}

type Status int

func (s *Status) setStatus(newStatus Status) {
	s = &newStatus
}

func main() {
	p := Person{Name : "Alice"}
	p.updateName("Bob")
	fmt.Println("Name:=", p.Name)

	var st Status = 10
	st.setStatus(20)
	
	fmt.Println("newStatus = ", st)

	var failedUpdate = func (g *int) { 
		x := 10; g = &x 
	}
	x := 5
	failedUpdate(&x)
	fmt.Println(x)

}



Cell[18]: Line 28 
 /var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_f7e15773/main.go:29:7: expected '(', found failedUpdate
 
	
	fmt.Println("newStatus = ", st)

 func failedUpdate(g *int) { 
 x := 10; g = &x 
	}

ERROR: parsing go files in TempDir "/var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_f7e15773": /var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_f7e15773/main.go:29:7: expected '(', found failedUpdate